In [2]:
import json
import csv
import numpy as np
import torch
import torch.nn as nn
from torch.utils import data
from tqdm import tqdm_notebook
import torch.optim as optim

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
with open('../../data/processed/masked_full_transcripts.json', 'r') as inp:
    transcripts = json.load(inp)

## Split Data

In [ ]:
np.random.shuffle(transcripts)

In [ ]:
train_split = len(transcripts)*8//10
valid_split = len(transcripts)*1//10
train = transcripts[:train_split]
valid = transcripts[train_split:train_split+valid_split]
test = transcripts[train_split+valid_split:]

train_csv = [('id', 'transcript', 'post_high')]
train_stock = []
for example in train:
    train_csv.append((example['id'], example['transcript'], float(example['post_high_low'][0])))
    historical = map(lambda x: x[1], sorted(example['historical_info'], key=lambda x: x[0]))
    train_stock.append(list(map(lambda x: list(map(float, x)), historical)))
    train_stock[-1] += [example['market_cap'], float(example['post_high_low'][0])]

valid_csv = [('id', 'transcript', 'post_high')]
valid_stock = []
for example in valid:
    valid_csv.append((example['id'], example['transcript'], float(example['post_high_low'][0])))
    historical = map(lambda x: x[1], sorted(example['historical_info'], key=lambda x: x[0]))
    valid_stock.append(list(map(lambda x: list(map(float, x)), historical)))
    valid_stock[-1] += [example['market_cap'], float(example['post_high_low'][0])]

test_csv = [('id', 'transcript', 'post_high')]
test_stock = []
for example in test:
    test_csv.append((example['id'], example['transcript'], float(example['post_high_low'][0])))
    historical = map(lambda x: x[1], sorted(example['historical_info'], key=lambda x: x[0]))
    test_stock.append(list(map(lambda x: list(map(float, x)), historical)))
    test_stock[-1] += [example['market_cap'], float(example['post_high_low'][0])]

In [ ]:
with open('../../data/processed/splits/train/transcripts.csv', 'w') as out:
    writer = csv.writer(out)
    header = None
    for row in train_csv:
        if header is None:
            writer.writerow(row)
            header = row
        else:
            [id, transcript, label] = row
            writer.writerow([id, ' '.join(transcript), label])

with open('../../data/processed/splits/valid/transcripts.csv', 'w') as out:
    writer = csv.writer(out)
    header = None
    for row in valid_csv:
        if header is None:
            writer.writerow(row)
            header = row
        else:
            [id, transcript, label] = row
            writer.writerow([id, ' '.join(transcript), label])

with open('../../data/processed/splits/test/transcripts.csv', 'w') as out:
    writer = csv.writer(out)
    header = None
    for row in test_csv:
        if header is None:
            writer.writerow(row)
            header = row
        else:
            [id, transcript, label] = row
            writer.writerow([id, ' '.join(transcript), label])

In [ ]:
with open('../../data/processed/splits/train/stock_data.json', 'w') as out:
    json.dump(train_stock, out)
with open('../../data/processed/splits/valid/stock_data.json', 'w') as out:
    json.dump(valid_stock, out)
with open('../../data/processed/splits/test/stock_data.json', 'w') as out:
    json.dump(test_stock, out)

In [ ]:
with open('../../data/processed/splits/valid/stock_data.csv', 'w') as out:
    writer = csv.writer(out)
    for row in valid_stock:
        writer.writerow(row)

## Load Data

In [5]:
with open('../../data/processed/splits/train/stock_data.json', 'r') as inp:
    train_stock = json.load(inp)
with open('../../data/processed/splits/valid/stock_data.json', 'r') as inp:
    valid_stock = json.load(inp)
with open('../../data/processed/splits/test/stock_data.json', 'r') as inp:
    test_stock = json.load(inp)

In [6]:
train_transcripts = []
with open('../../data/processed/splits/train/transcripts.csv', 'r') as inp:
    reader = csv.reader(inp)
    headers = None
    for row in reader:
        if headers is None:
            headers = row
        else:
            row[1] = row[1].split(' ')
            train_transcripts.append(row)

In [30]:
len(stock_train)

NameError: name 'stock_train' is not defined

## Create Dataset

In [6]:
class StockDataset(data.Dataset):
    def __init__(self, examples):
        examples = np.array(examples)
        self.labels = examples[:,-1]
        self.market_cap = examples[:,-2]
        self.examples = np.array(examples[:,:-2].tolist())

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.examples)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Load data and get label
        X = torch.tensor(self.examples[index])
        auxiliary = self.market_cap[index]
        y = self.labels[index]

        return X, auxiliary, y

In [7]:
valid_dataset = StockDataset(valid_stock)

In [8]:
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}
valid_generator = data.DataLoader(valid_dataset, **params)

## Train Model

In [12]:
class BaselineStockPredictor(nn.Module):
    """
    Model that will read in plain stock ticker values over time and decide whether to buy, sell, or hold at the current price.
    """
    def __init__(self, num_series_features=1, num_auxiliary_features=1, hidden_size=128, output_size=1):
        """
        Attributes:
            num_series_features: the size of the feature set for an individual
                                 stock price example (e.g. if we include high,
                                 low, average, num_series_features will equal 3
            num_auxiliary_features: the number of auxiliary (not dependent on time)
                                    features we are adding (e.g. if we include the 1yr
                                    high and the market capitalization, num_auxiliary_features
                                    would equal 2
            output_size: the size of the outputted vector. For evaluation, we would use a
                         size of 1 (stock price) or 3 (buy, sell, hold classification).
                         For use in the looking glass model, we want an encoding so we might
                         use a size of 128 to feed into the model.
        """
        super().__init__()
        self.recurrent = nn.LSTM(
            input_size=num_series_features,
            hidden_size=hidden_size,
            num_layers=1,
            bidirectional=False,
            batch_first=True,
            dropout=0.5
        )
        # concatenate LSTM output with auxiliary features
        # output predicted price
        self.linear1 = nn.Linear(hidden_size+num_auxiliary_features, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        self.init_weights()

    def init_weights(self):
        """
        Initializes the weights of the model
        """
        for layer in [self.linear1, self.linear2]:
            nn.init.xavier_uniform_(layer.weight)
            nn.init.constant_(layer.bias, 0.0)

    def forward(self, X_series, X_auxiliary):
        """
        Moves the model through each layer
        Parameters:
            X_series: an [N, num_series_examples, num_series_features] size vector
                      where N is the batch size, num_series_examples is how many stock prices
                      we are providing per example (e.g. weekly for the last 3 months), and
                      num_series_features is the same as described in __init__
            X_auxiliary: an [N, num_auxiliary_features] vector
        """
        recurrent_output,_ = self.recurrent(X_series)
        recurrent_output = torch.mean(recurrent_output, 1)
        # We might need this
        # recurrent_output = torch.squeeze(1) 
        aux_combined = torch.cat([recurrent_output, X_auxiliary], dim=1)
        z1 = self.linear1(aux_combined)
        a1 = nn.functional.relu(z1)
        output = self.linear2(a1)

        return output

In [13]:
def get_stock_iterator(input_data, batch_size, train=True, shuffle=True):
    dataset = StockDataset(input_data)
    iterator = data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=5)
    return iterator
    
def train_model(train, valid, num_epochs=200, learning_rate=0.003, existing_model=None):
    batch_size = 128
    train_iterator = get_stock_iterator(train, batch_size)
    valid_iterator = get_stock_iterator(valid, batch_size)
    
    model = None
    
    if existing_model is None:
        model = BaselineStockPredictor(num_series_features=2, hidden_size=64)
    else:
        model = existing_model
    model = model.float().cuda()
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), learning_rate)
    
    losses = []
    valid_scores = []
    
    min_mse = float('inf')
    delay = 0
    MAX_INC = 20
    
    for epoch in tqdm_notebook(range(num_epochs)):
        model.train()
        iter_losses = []
        for batch_series, batch_aux, batch_labels in train_iterator:
            batch_aux = torch.reshape(batch_aux, (-1,1))
            optimizer.zero_grad()
            outputs = model(batch_series.float().cuda(), batch_aux.float().cuda())
            batch_labels = torch.reshape(batch_labels, (-1,1))
            loss = criterion(outputs, batch_labels.float().cuda())
            loss.backward()
            optimizer.step()
            iter_losses.append(loss.item())
        iter_losses = np.array(iter_losses)
        losses.append(np.mean(iter_losses))
        
        valid_mse = []
        model.eval()

        for valid_batch_series, valid_batch_aux, valid_batch_labels in valid_iterator:
            valid_batch_aux = torch.reshape(valid_batch_aux, (-1,1))
            outputs = model(valid_batch_series.float().cuda(), valid_batch_aux.float().cuda())
            valid_batch_labels = torch.reshape(valid_batch_labels, (-1,1))
            loss = criterion(outputs, valid_batch_labels.float().cuda())
            valid_mse.append(loss.item())
        valid_mse = np.mean(valid_mse)
        print(f'Completed epoch {epoch}. Valid MSE: {valid_mse}')


        if valid_mse < min_mse:
            min_mse = valid_mse
            delay = 0
            torch.save(model, 'model.ckpt')
        else:
            delay += 1
    
    return model, losses

In [18]:
model, losses = train_model(train_stock, valid_stock, num_epochs=6000)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Completed epoch 0. Valid MSE: 29669.91015625
Completed epoch 1. Valid MSE: 27075.9296875
Completed epoch 2. Valid MSE: 24932.22265625
Completed epoch 3. Valid MSE: 23504.216796875
Completed epoch 4. Valid MSE: 22454.732421875
Completed epoch 5. Valid MSE: 22103.619140625
Completed epoch 6. Valid MSE: 22101.65625
Completed epoch 7. Valid MSE: 22045.361328125
Completed epoch 8. Valid MSE: 21904.498046875
Completed epoch 9. Valid MSE: 21628.341796875
Completed epoch 10. Valid MSE: 20893.59765625
Completed epoch 11. Valid MSE: 20415.623046875
Completed epoch 12. Valid MSE: 19469.298828125
Completed epoch 13. Valid MSE: 18786.96875
Completed epoch 14. Valid MSE: 17901.986328125
Completed epoch 15. Valid MSE: 17261.041015625
Completed epoch 16. Valid MSE: 16414.109375
Completed epoch 17. Valid MSE: 15585.60546875
Completed epoch 18. Valid MSE: 14973.7529296875
Completed epoch 19. Valid MSE: 14605.181640625
Completed epoch 20. Valid MSE: 13576.9931640625
Completed epoch 21. Valid MSE: 12679.9

Completed epoch 168. Valid MSE: 391.7317199707031
Completed epoch 169. Valid MSE: 467.5809326171875
Completed epoch 170. Valid MSE: 544.4385375976562
Completed epoch 171. Valid MSE: 372.1713562011719
Completed epoch 172. Valid MSE: 266.9889221191406
Completed epoch 173. Valid MSE: 268.0270690917969
Completed epoch 174. Valid MSE: 231.5167999267578
Completed epoch 175. Valid MSE: 308.8145446777344
Completed epoch 176. Valid MSE: 279.77593994140625
Completed epoch 177. Valid MSE: 257.26226806640625
Completed epoch 178. Valid MSE: 750.3549194335938
Completed epoch 179. Valid MSE: 565.10205078125
Completed epoch 180. Valid MSE: 3413.273681640625
Completed epoch 181. Valid MSE: 2240.64453125
Completed epoch 182. Valid MSE: 2689.3564453125
Completed epoch 183. Valid MSE: 713.2313232421875
Completed epoch 184. Valid MSE: 441.62225341796875
Completed epoch 185. Valid MSE: 360.5782775878906
Completed epoch 186. Valid MSE: 203.63323974609375
Completed epoch 187. Valid MSE: 165.9014129638672
Comp

Completed epoch 333. Valid MSE: 325.98687744140625
Completed epoch 334. Valid MSE: 390.2200927734375
Completed epoch 335. Valid MSE: 335.4064636230469
Completed epoch 336. Valid MSE: 350.34979248046875
Completed epoch 337. Valid MSE: 270.6438903808594
Completed epoch 338. Valid MSE: 327.99578857421875
Completed epoch 339. Valid MSE: 312.72503662109375
Completed epoch 340. Valid MSE: 319.18988037109375
Completed epoch 341. Valid MSE: 301.8866882324219
Completed epoch 342. Valid MSE: 320.1248779296875
Completed epoch 343. Valid MSE: 269.0160217285156
Completed epoch 344. Valid MSE: 328.8674621582031
Completed epoch 345. Valid MSE: 307.3306884765625
Completed epoch 346. Valid MSE: 339.32464599609375
Completed epoch 347. Valid MSE: 356.5552978515625
Completed epoch 348. Valid MSE: 333.62127685546875
Completed epoch 349. Valid MSE: 326.2830810546875
Completed epoch 350. Valid MSE: 327.7666015625
Completed epoch 351. Valid MSE: 285.0477294921875
Completed epoch 352. Valid MSE: 331.1245117187

Completed epoch 497. Valid MSE: 776.1412353515625
Completed epoch 498. Valid MSE: 561.503662109375
Completed epoch 499. Valid MSE: 445.8585205078125
Completed epoch 500. Valid MSE: 473.79437255859375
Completed epoch 501. Valid MSE: 542.8885498046875
Completed epoch 502. Valid MSE: 572.6483154296875
Completed epoch 503. Valid MSE: 542.5585327148438
Completed epoch 504. Valid MSE: 524.5477905273438
Completed epoch 505. Valid MSE: 558.7437744140625
Completed epoch 506. Valid MSE: 559.144287109375
Completed epoch 507. Valid MSE: 559.3298950195312
Completed epoch 508. Valid MSE: 531.6336669921875
Completed epoch 509. Valid MSE: 548.1639404296875
Completed epoch 510. Valid MSE: 540.3116455078125
Completed epoch 511. Valid MSE: 548.9262084960938
Completed epoch 512. Valid MSE: 527.312255859375
Completed epoch 513. Valid MSE: 513.3230590820312
Completed epoch 514. Valid MSE: 528.5159301757812
Completed epoch 515. Valid MSE: 542.9752197265625
Completed epoch 516. Valid MSE: 527.4623413085938
Co

...

Completed epoch 5206. Valid MSE: 218.5465545654297
Completed epoch 5207. Valid MSE: 233.102783203125
Completed epoch 5208. Valid MSE: 225.7519073486328
Completed epoch 5209. Valid MSE: 222.50906372070312
Completed epoch 5210. Valid MSE: 228.130126953125
Completed epoch 5211. Valid MSE: 253.5665740966797
Completed epoch 5212. Valid MSE: 412.40185546875
Completed epoch 5213. Valid MSE: 352.1471862792969
Completed epoch 5214. Valid MSE: 383.0326843261719
Completed epoch 5215. Valid MSE: 195.39657592773438
Completed epoch 5216. Valid MSE: 278.04144287109375
Completed epoch 5217. Valid MSE: 209.04718017578125
Completed epoch 5218. Valid MSE: 261.990966796875
Completed epoch 5219. Valid MSE: 230.37619018554688
Completed epoch 5220. Valid MSE: 212.5738067626953
Completed epoch 5221. Valid MSE: 235.90121459960938
Completed epoch 5222. Valid MSE: 199.425048828125
Completed epoch 5223. Valid MSE: 304.9893493652344
Completed epoch 5224. Valid MSE: 218.26380920410156
Completed epoch 5225. Valid MS

Completed epoch 5366. Valid MSE: 218.683837890625
Completed epoch 5367. Valid MSE: 280.5534973144531
Completed epoch 5368. Valid MSE: 178.82843017578125
Completed epoch 5369. Valid MSE: 231.67031860351562
Completed epoch 5370. Valid MSE: 163.39718627929688
Completed epoch 5371. Valid MSE: 191.2700958251953
Completed epoch 5372. Valid MSE: 216.7708740234375
Completed epoch 5373. Valid MSE: 213.49581909179688
Completed epoch 5374. Valid MSE: 242.88958740234375
Completed epoch 5375. Valid MSE: 164.41281127929688
Completed epoch 5376. Valid MSE: 169.98194885253906
Completed epoch 5377. Valid MSE: 168.6617889404297
Completed epoch 5378. Valid MSE: 172.02630615234375
Completed epoch 5379. Valid MSE: 211.3204803466797
Completed epoch 5380. Valid MSE: 154.45359802246094
Completed epoch 5381. Valid MSE: 170.41278076171875
Completed epoch 5382. Valid MSE: 256.0426330566406
Completed epoch 5383. Valid MSE: 227.33815002441406
Completed epoch 5384. Valid MSE: 217.15223693847656
Completed epoch 5385

Completed epoch 5526. Valid MSE: 232.76181030273438
Completed epoch 5527. Valid MSE: 226.3066864013672
Completed epoch 5528. Valid MSE: 263.3293151855469
Completed epoch 5529. Valid MSE: 246.57005310058594
Completed epoch 5530. Valid MSE: 271.50799560546875
Completed epoch 5531. Valid MSE: 221.82061767578125
Completed epoch 5532. Valid MSE: 257.5815124511719
Completed epoch 5533. Valid MSE: 219.60552978515625
Completed epoch 5534. Valid MSE: 228.2445526123047
Completed epoch 5535. Valid MSE: 220.36997985839844
Completed epoch 5536. Valid MSE: 271.24371337890625
Completed epoch 5537. Valid MSE: 226.18360900878906
Completed epoch 5538. Valid MSE: 249.638427734375
Completed epoch 5539. Valid MSE: 218.727783203125
Completed epoch 5540. Valid MSE: 240.59107971191406
Completed epoch 5541. Valid MSE: 235.98968505859375
Completed epoch 5542. Valid MSE: 222.3810272216797
Completed epoch 5543. Valid MSE: 246.2044677734375
Completed epoch 5544. Valid MSE: 505.4119873046875
Completed epoch 5545. V

Completed epoch 5687. Valid MSE: 361.3183288574219
Completed epoch 5688. Valid MSE: 325.1945495605469
Completed epoch 5689. Valid MSE: 460.61480712890625
Completed epoch 5690. Valid MSE: 398.6413269042969
Completed epoch 5691. Valid MSE: 1640.05810546875
Completed epoch 5692. Valid MSE: 556.7377319335938
Completed epoch 5693. Valid MSE: 2107.419677734375
Completed epoch 5694. Valid MSE: 403.15679931640625
Completed epoch 5695. Valid MSE: 495.7575378417969
Completed epoch 5696. Valid MSE: 760.8382568359375
Completed epoch 5697. Valid MSE: 488.09393310546875
Completed epoch 5698. Valid MSE: 692.6553955078125
Completed epoch 5699. Valid MSE: 374.42352294921875
Completed epoch 5700. Valid MSE: 375.7945251464844
Completed epoch 5701. Valid MSE: 574.2725219726562
Completed epoch 5702. Valid MSE: 390.6096496582031
Completed epoch 5703. Valid MSE: 385.8209533691406
Completed epoch 5704. Valid MSE: 460.0759582519531
Completed epoch 5705. Valid MSE: 374.9566955566406
Completed epoch 5706. Valid 

Completed epoch 5848. Valid MSE: 169.77996826171875
Completed epoch 5849. Valid MSE: 171.5479278564453
Completed epoch 5850. Valid MSE: 152.6156463623047
Completed epoch 5851. Valid MSE: 182.37625122070312
Completed epoch 5852. Valid MSE: 179.8022003173828
Completed epoch 5853. Valid MSE: 174.2165985107422
Completed epoch 5854. Valid MSE: 171.9759979248047
Completed epoch 5855. Valid MSE: 174.15866088867188
Completed epoch 5856. Valid MSE: 173.22271728515625
Completed epoch 5857. Valid MSE: 169.87445068359375
Completed epoch 5858. Valid MSE: 165.7123565673828
Completed epoch 5859. Valid MSE: 207.95005798339844
Completed epoch 5860. Valid MSE: 206.6595001220703
Completed epoch 5861. Valid MSE: 160.34214782714844
Completed epoch 5862. Valid MSE: 173.99998474121094
Completed epoch 5863. Valid MSE: 185.20130920410156
Completed epoch 5864. Valid MSE: 190.78558349609375
Completed epoch 5865. Valid MSE: 165.7781219482422
Completed epoch 5866. Valid MSE: 167.85848999023438
Completed epoch 5867

In [16]:
def eval_model(model, test):
    batch_size = 128
    test_iterator = get_stock_iterator(test, batch_size, shuffle=False)
        
    criterion = nn.MSELoss()
    
    model.eval()
    losses = []
    for batch_series, batch_aux, batch_labels in test_iterator:
        batch_aux = torch.reshape(batch_aux, (-1,1))
        outputs = model(batch_series.float().cuda(), batch_aux.float().cuda())
        batch_labels = torch.reshape(batch_labels, (-1,1))
        loss = criterion(outputs, batch_labels.float().cuda())
        losses.append(loss.item())
    return np.mean(losses)

In [25]:
eval_model(model, test_stock)

587.68408203125

In [14]:
with open('stock_model_losses_10k_epochs.json', 'w') as out:
    json.dump(losses, out)

In [21]:
with open('../../data/models/stock_6k_2l.json', 'w') as out:
    json.dump(losses, out)

In [26]:
torch.save(model, '../../data/models/stock_6k.pt')

In [27]:
baseline_model = torch.load('../../data/models/stock_6k.pt')

In [28]:
eval_model(baseline_model, test_stock)

587.68408203125

In [29]:
test_stock[0]

[[46.26, 45.2],
 [45.59, 44.77],
 [45.24, 44.8],
 [45.91, 44.96],
 [45.95, 45.2],
 233.16,
 45.57]